In [2]:
import numpy as np
import pandas as pd

In [72]:
smd_train_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_train.npy")
smd_test_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_test.npy")
smd_label_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_test_label.npy")

In [73]:
smd_train_data.shape, smd_test_data.shape, smd_label_data.shape

((708405, 38), (708420, 38), (708420,))

In [59]:
sample_df = pd.read_csv("results.csv")
sample_df_flag = sample_df.copy()

for column in sample_df.columns:
    if column == "measDate": continue
    # Create a new column with '_flag' suffix that checks for NaN
    sample_df_flag[column + '_NaN'] = sample_df[column].isna().astype(float)

for column in sample_df_flag.columns:
    if column == "measDate": continue
    # Create a new column with '_flag' suffix that checks for NaN
    sample_df_flag[column] = pd.to_numeric(sample_df_flag[column], errors="coerce")
    
sample_df_flag_fillnan = sample_df_flag.fillna(0)

np_time_stamps = sample_df_flag_fillnan["measDate"].to_numpy()

sample_df_flag_fillnan_onlydata = sample_df_flag_fillnan.drop(columns=['measDate'], inplace=False)
np_df_flat_fillna = sample_df_flag_fillnan_onlydata.to_numpy()

In [60]:
sample_df_flag.describe()

,CI,DT,HI,ID,IR,IS,TI,HL,PL,WC,...,ID_NaN,IR_NaN,IS_NaN,TI_NaN,HL_NaN,PL_NaN,WC_NaN,HE_NaN,HR_NaN,RP_NaN
count,2049.000000,0.0,2054.000000,0.0,0.0,0.0,2040.000000,1477.000000,0.0,0.0,...,2083.0,2083.0,2083.0,2083.000000,2083.000000,2083.0,2083.0,2083.0,2083.0,2083.000000
mean,406.633367,NaN,98.805078,NaN,NaN,NaN,18.507789,5.125037,NaN,NaN,...,1.0,1.0,1.0,0.020643,0.290927,1.0,1.0,1.0,1.0,0.197792
std,133.296646,NaN,1.813378,NaN,NaN,NaN,3.918314,0.849521,NaN,NaN,...,0.0,0.0,0.0,0.142221,0.454299,0.0,0.0,0.0,0.0,0.398430
min,1.270000,NaN,85.180000,NaN,NaN,NaN,11.670000,2.320000,NaN,NaN,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000
25%,289.230000,NaN,98.760000,NaN,NaN,NaN,15.527500,4.730000,NaN,NaN,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000
50%,385.540000,NaN,99.430000,NaN,NaN,NaN,17.465000,5.000000,NaN,NaN,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000
75%,517.350000,NaN,99.830000,NaN,NaN,NaN,20.582500,6.000000,NaN,NaN,...,1.0,1.0,1.0,0.000000,1.000000,1.0,1.0,1.0,1.0,0.000000
max,812.180000,NaN,100.000000,NaN,NaN,NaN,32.730000,7.000000,NaN,NaN,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.000000


In [61]:
sample_df.to_csv("results_input.csv")
sample_df_flag.to_csv("results_flag.csv")
sample_df_flag_fillnan.to_csv('results_flag_fillna.csv')

In [62]:
def positional_encoding(time_steps, dimension):
    assert dimension % 2 == 0, "Dimension must be even."
    position = np.arange(time_steps).reshape(-1, 1)
    div_term = np.exp(np.arange(0, dimension, 2) * -(np.log(10000.0) / dimension))
    pe = np.zeros((time_steps, dimension))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    return pe

# Specify dimension based on your model requirements
dimension = 14 # num of predefined columns (We will use only up to 13)
pos_encodings = positional_encoding(365 * 24, dimension) # 8760 = 365 days * 24 hours

In [63]:
from datetime import datetime

def get_hour_index_of_year(timestamp):
    # Parse the timestamp string to a datetime object
    date_time_obj = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    
    # Calculate the day of the year from the datetime object
    day_of_year = date_time_obj.timetuple().tm_yday
    
    # Get the hour from the datetime object
    hour_of_day = date_time_obj.hour
    
    # Calculate the hour index within the year (0-indexed)
    hour_index_of_year = (day_of_year - 1) * 24 + hour_of_day
    
    return hour_index_of_year

In [64]:
for i, time_stamp in enumerate(np_time_stamps):
    tidx = get_hour_index_of_year(time_stamp)
    np_df_flat_fillna[i, :13] += pos_encodings[tidx, :13]    

In [69]:
np_df_flat_fillna[4]

array([-0.82183501, -0.56972556, 95.26263333, -0.93576158,  0.42414733,
        0.9055932 , -0.89521588,  0.44563274,  0.98335725,  0.18168246,
        0.36383781,  0.93146232,  0.09973343,  1.        ,  1.        ,
        0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.        ])

In [58]:
pos_encodings[0]

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [70]:
np.savez("sample_data.npz", np_df_flat_fillna)

In [71]:
np_df_flat_fillna.shape

(2083, 26)

In [5]:
data = np.load("/home/joono/Anomaly-Transformer/dataset/SmartFarm/smartfarm_train.npz")

In [13]:
data['arr_0']

array([[ 6.06276017e+02,  9.94364266e-01,  9.37605845e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 8.94010170e-01,  4.48046667e-01,  9.60588643e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 6.46360054e+02, -5.10202971e-01,  9.55614104e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [ 5.67344369e+02,  9.14596145e-01,  9.94832508e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 4.36178087e+02,  1.53894037e-01,  9.78326470e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 6.75573363e+02, -7.48297539e-01,  9.91062075e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00]])